In [18]:
import pandas as pd
import numpy as np
import re
import html
from number_parser import parse_ordinal


In [19]:
df = pd.read_csv("Open_Restaurant_Applications__Historic__20231202_t.csv")

In [20]:
df.shape

(14428, 35)

In [21]:
pd.set_option("display.max_columns", None)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14428 entries, 0 to 14427
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   objectid                                  14428 non-null  int64  
 1   globalid                                  14428 non-null  object 
 2   Seating Interest (Sidewalk/Roadway/Both)  14428 non-null  object 
 3   Restaurant Name                           14427 non-null  object 
 4   Legal Business Name                       14427 non-null  object 
 5   Doing Business As (DBA)                   14416 non-null  object 
 6   Building Number                           12776 non-null  object 
 7   Street                                    14428 non-null  object 
 8   Borough                                   14428 non-null  object 
 9   Postcode                                  14428 non-null  int64  
 10  Business Address                  

In [23]:
# checking numerical columns in the dataset

numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]

total_numerical_cols = len(df.select_dtypes(numerics).columns)
total_numerical_cols

# there are 15 numerical columns and 25 non-numerical columns

15

In [24]:
df.describe()

,objectid,Postcode,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL
count,14428.000000,14428.000000,12118.000000,12118.000000,12118.000000,9054.000000,9054.000000,9054.000000,13121.000000,13121.000000,13121.000000,13121.000000,13121.000000,1.302600e+04,1.302600e+04
mean,7266.598766,10594.626074,32.959894,10.132283,372.778099,30.820853,8.205324,259.750497,40.733412,-73.953890,5.218581,17.203948,2928.269187,2.248597e+06,2.194042e+09
std,4184.357841,922.296122,34.496100,6.954057,926.289956,26.737258,3.364510,401.388896,0.056158,0.059173,3.635352,15.279790,12812.777168,1.308012e+06,1.273212e+09
min,32.000000,7002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.511019,-74.248014,1.000000,1.000000,1.000000,1.000000e+06,0.000000e+00
25%,3641.750000,10016.000000,15.000000,6.000000,108.000000,18.000000,8.000000,128.000000,40.707473,-73.990251,2.000000,3.000000,69.000000,1.025026e+06,1.008400e+09
50%,7266.500000,10303.000000,24.000000,8.000000,206.000000,24.000000,8.000000,192.000000,40.734734,-73.970537,5.000000,11.000000,149.000000,2.003088e+06,2.024115e+09
75%,10893.250000,11220.000000,40.000000,13.000000,400.000000,36.000000,8.000000,296.000000,40.761872,-73.926113,7.000000,33.000000,466.000000,3.212196e+06,3.054050e+09
max,14516.000000,91206.000000,1000.000000,50.000000,50000.000000,1000.000000,50.000000,25000.000000,40.911190,-73.702668,81.000000,51.000000,157903.000000,5.169029e+06,5.080430e+09


In [25]:
# chcking the percent of null values in each column

round(df.isnull().sum().sort_values(ascending=False) / len(df) * 100, 2)

landmarkDistrict_terms                      85.24
Roadway Dimensions (Area)                   37.25
Roadway Dimensions (Length)                 37.25
Roadway Dimensions (Width)                  37.25
SLA Serial Number                           34.52
SLA License Type                            34.51
Sidewalk Dimensions (Length)                16.01
Sidewalk Dimensions (Width)                 16.01
Sidewalk Dimensions (Area)                  16.01
Building Number                             11.45
BIN                                          9.72
BBL                                          9.72
Census Tract                                 9.06
Council District                             9.06
Community Board                              9.06
Latitude                                     9.06
Longitude                                    9.06
NTA                                          9.06
Doing Business As (DBA)                      0.08
Food Service Establishment Permit #          0.04


# Changes needed to be done
- Drop landmarkDistrict_terms column
- Remove {} from the rows of globalid column 
- Change Dtype of  Time of Submission column
- Remove html tags from the columns
- Make columns title
- Make business entity abbreviations in correct format (from Llc to LLC and Inc... to Inc. etc)
- Make NY upper case again (which became title due to previous step)
- Give proper suffix to digits in the columns (61St to 61st)
- Replace the Nan and undefiend values in the Building number column with actual building number
- Remove the building number from street column
- SLA column should have a serial number but instead some rows have a month-number like(july-79) etc (I think it should be dropped)


### Dropping the landmarkDistrict_terms column

In [26]:
df = df.drop(["landmarkDistrict_terms"], axis=1)

### Removing {} from the rows of globalid column

In [27]:
df["globalid"] = df["globalid"].str.strip("{}")

### Chainging Dtype of Time of Submission column

In [28]:
df["Time of Submission"] = pd.to_datetime(df["Time of Submission"])

### function to remove the html tags from the values in column


In [29]:
def html_content_remover(column):
    return column.apply(lambda x: html.unescape(str(x)))

### removing html tags form various columns

In [30]:
df["Restaurant Name"] = html_content_remover(df["Restaurant Name"])
df["Doing Business As (DBA)"] = html_content_remover(df["Doing Business As (DBA)"])
df["Legal Business Name"] = html_content_remover(df["Legal Business Name"])


### function to make title of all the values in the column and correct the problems that arise due to title


In [31]:
def make_title_and_corrections(df, column):
    df[column] = df[column].str.title()
    
    df[column] = df[column].fillna("")
    
    pattern = r'\b(\d+)(Th|Nd|St|Rd)\b'
    pattern_1 = r'\b([A-Za-z]+)(\'[A-Z])\b'
    corp_pattern = r'\bCorp(?:oration|orations)?(?![.])\b'
    inc_pattern = r'\bInc(?:orporated)?(?![.])\b'

    df[column] = df[column].apply(lambda x : re.sub(pattern, lambda m: m.group(1) + m.group(2).lower(), x))
    df[column] = df[column].apply(lambda x: re.sub(pattern_1,lambda m : m.group(1) + m.group(2).lower(),x ))
    
    df[column] = df[column].str.replace("Llc","LLC")
    df[column] = df[column].str.replace(inc_pattern,"Inc.",regex=True)
    df[column] = df[column].str.replace(corp_pattern,"Corp.",regex=True)
    


In [32]:
make_title_and_corrections(df, "Restaurant Name")
make_title_and_corrections(df, "Legal Business Name")
make_title_and_corrections(df, "Doing Business As (DBA)")



### Function to fill the Building Number column(replace NaN and undefiend values with the building numbers)

In [33]:
def fill_building_number(dataframe, business_address: str, building_number: str):
    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        # checking if this at cuurent row building number is None or 'undefined'
        if pd.isna(row[building_number]) or row[building_number] == "undefined":

            first_word = row[business_address].split()[0]
            # if it is then split the business address at current index and take the first object and put it in first_word variable

            df.at[index, building_number] = first_word
            # Assigning the value of first_word variable to the building number at that row


fill_building_number(df, "Business Address", "Building Number")

### Function to remove building number from  Street column

In [34]:
def street_transformation(dataframe, street_col, building_number_col):
    '''This function removes Building number from the street column'''

    for index, row in df.iterrows():

        # storing the first word of Street column at each row in first_word variable
        first_word = row[street_col].split()[0]

        # checking if building is equal to the first_word
        if row[building_number_col] == first_word:
            new_street = " ".join(row[street_col].split()[1:])
            # splitting the street at each row and indexing from 1 then joining it

            df.at[index, street_col] = new_street
            # replacing the value of street at a specific index with the value of new_street

street_transformation(df, "Street", "Building Number")

### Making Ny upper case again accross the Business Address column


In [35]:
df["Business Address"] = df["Business Address"].str.replace(r"\bNy\b", "NY", regex=True)

-------------------- =============== --------------------------------------

## Changes to be made in street column

- Capitalize the street column 
- if there are suffix behind street numbers make them proper like in column [12660, 10488, ] 
- if the steet number don't have suffix the add proper suffix to them [6391, 11583,12314] 
- Ave to Avenue, Boulevard to Blvd , St., to Street if St in last  [7114,5934,12021] 
- Words to digits [12021, ] 

In [36]:
def column_tansformation(df,column):
    df[column] = df[column].str.title()
    
    
    def what(x):

        def is_ordinal(word):
            ordinal_words = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eighth', 'Ninth', 'Tenth']
            return word in ordinal_words

        l = x.split()

        for i in range(len(l)):
            if is_ordinal(l[i]):
                parsed_word = parse_ordinal(l[i])
                l[i] = str(parsed_word)

        return " ".join(l)
            
    
    
    def ordinal(m):
        n = int(m.group())
        if 5 <= n <= 20 :
            suffix = 'th'
        else:
            remainder = n % 10
            if remainder == 1:
                suffix = 'st'
            elif remainder == 2:
                suffix = 'nd'
            elif remainder == 3:
                suffix = 'rd'
            else:
                suffix = 'th'
        output = str(n) + suffix
        return output
    
    pattern = r'\b(\d+)(Th|Nd|St|Rd)\b'
    ave_pattern = r'\bAve\.?$|\bAve\b|\bAv\b'
    st_pattern = r'\bSt\.?$\b|\bSt\b'
    blvd_pattern = r'\bBlvd\b'
    number = r'\b\d+\b'
    
    
    df[column] = df[column].apply(lambda x : re.sub(pattern, lambda m: m.group(1) + m.group(2).lower(), str(x)))
    df[column] = df[column].str.replace(ave_pattern, 'Avenue', regex = True)
    df[column] = df[column].str.replace(st_pattern, 'Street', regex = True)
    df[column] = df[column].str.replace(blvd_pattern, 'Boulevard', regex = True)
    df[column] = df[column].apply(what)
    df[column] = df[column].str.replace(re.compile(number), lambda x: ordinal(x), regex=True)
    

    
    
    
column_tansformation(df,"Street")

## Combining the Building number + Street + Borough columns + NY = Business Address

In [37]:
df["Business Address"] = df.apply(lambda row: f"{row['Building Number']}, {row['Street']}, {row['Borough']}, NY", axis=1)

In [38]:
df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318, East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965,TW,no,yes,2022-06-09 16:26:00,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,Street. Nicholas Avenue,Manhattan,10026,"55, Street. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526,RW,no,yes,2020-06-26 20:38:00,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,sidewalk,Americas Cafe & Grill,68th Grill Inc.,Americas Cafe & Grill,1159,3rd Avenue,Manhattan,10065,"1159, 3rd Avenue, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,yes,2021-10-22 11:01:00,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23rd Street,Manhattan,10011,"208, West 23rd Street, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,Aug-82,OP,no,yes,2020-12-14 19:54:00,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190, Orchard Street, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,Jul-61,TW,no,yes,2021-12-22 12:46:00,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425, College Point Boulevard, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,Feb-05,OP,no,yes,2020-08-04 15:27:00,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61st Street,Brooklyn,11220,"752B, 61st Street, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,yes,2020-07-13 15:53:00,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,476f88d2-d3e8-4e83-8686-8bac2f752cf5,sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10, Murray Street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882,RW,no,yes,2020-06-21 13:46:00,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,ff0d04ef-1d9a-47d6-8f20-8bdbdbce2792,sidewalk,Le Cafe Coffee,Le Cafe Coffee LLC,Le Cafe Coffee LLC,145,4th Avenue,Manhattan,10003,"145, 4th Avenue, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,yes,2020-06-19 13:19:00,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


In [39]:
### DON"T DELETE THIS CELL IT IS IMPORTANT

def ordinal(m: str):
        n = int(m.group())
        if 11 <= (n % 100) <= 13:
            suffix = 'th'
        else:
            suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
        output = str(n) + suffix
        return output


line = "this is 52 street 123"
number = r'\b\d+\b'
pattern = re.compile(number)
out = re.sub(pattern,lambda x : ordinal(x),line)
print(out)


this is 52nd street 123rd


### Handling values in SLA Serial Number Column (contains ###, and month-random_two_digit values)

In [40]:
series = (df["SLA Serial Number"].str.match(r"^[A-Za-z]+\-\d+|\#+") & (df["Qualify Alcohol"] == 'yes'))
df.loc[series, "SLA Serial Number"] = "TO BE DEFINED"


### Capitalizing Seating Interest (Sidewalk/Roadway/Both) column

In [41]:
df["Seating Interest (Sidewalk/Roadway/Both)"] = df['Seating Interest (Sidewalk/Roadway/Both)'].str.capitalize()

## Saving into csv

In [42]:
df.to_csv("Cleaned_ORA_historic.csv",index=False)